In [1]:
import re
import os
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
from sklearn import metrics
import jieba
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [2]:
# 中文字符显示
plt.rcParams['font.sans-serif']=['SimHei'] 
plt.rcParams['axes.unicode_minus']=False 

In [3]:
def load_dataset(fname):
    x = []
    y = []
    with open(fname, 'r', encoding='utf8') as f:
        for line in f:
            line = line.strip()
            label, content = line.split(maxsplit=1)
            x.append(content)
            y.append(label)
    return pd.DataFrame({
        'content': x,
        'label': y
    })

In [4]:
train = load_dataset('./data/cnews.train.txt')

In [5]:
test = load_dataset('./data/cnews.test.txt')

In [6]:
train

,content,label
0,马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳，国奥队依然没有...,体育
1,商瑞华首战复仇心切 中国玫瑰要用美国方式攻克瑞典多曼来了，瑞典来了，商瑞华首战求3分的信心也...,体育
2,冠军球队迎新欢乐派对 黄旭获大奖张军赢下PK赛新浪体育讯12月27日晚，“冠军高尔夫球队迎新...,体育
3,辽足签约危机引注册难关 高层威逼利诱合同笑里藏刀新浪体育讯2月24日，辽足爆发了集体拒签风波...,体育
4,揭秘谢亚龙被带走：总局电话骗局 复制南杨轨迹体坛周报特约记者张锐北京报道 谢亚龙已经被公安...,体育
...,...,...
49995,打好投资的决胜局□国泰基金 阿邰就长期而言，资产配置占投资成功与否的决定因素高达90%以上。...,财经
49996,昔明星基金今年首月负收益 QDII成今年胸闷基金□晨报记者 陈重博2009年以来，所有偏股型...,财经
49997,沪基指全周大涨8.25% 创两个月最大周涨幅全景网2月6日讯 受到A股市场牛年高歌猛进影响，...,财经
49998,沪基指半日涨2.01% 两市封基近乎全线上扬全景网2月6日讯 沪深基金指数周五早盘大幅收高，...,财经


In [7]:
train.shape

(50000, 2)

In [8]:
train['label'].value_counts()

体育    5000
娱乐    5000
家居    5000
房产    5000
教育    5000
时尚    5000
时政    5000
游戏    5000
科技    5000
财经    5000
Name: label, dtype: int64

In [9]:
test

,content,label
0,鲍勃库西奖归谁属？ NCAA最强控卫是坎巴还是弗神新浪体育讯如今，本赛季的NCAA进入到了末...,体育
1,麦基砍28+18+5却充满寂寞 纪录之夜他的痛阿联最懂新浪体育讯上天对每个人都是公平的，贾维...,体育
2,黄蜂vs湖人首发：科比冲击七连胜 火箭两旧将登场新浪体育讯北京时间3月28日，NBA常规赛洛...,体育
3,双面谢亚龙作秀终成做作 谁来为低劣行政能力埋单是谁任命了谢亚龙？谁放纵了谢亚龙？谁又该为谢亚...,体育
4,兔年首战山西换帅后有虎胆 张学文用乔丹名言励志今晚客场挑战浙江稠州银行队，是山西汾酒男篮的兔...,体育
...,...,...
9995,近期18只偏股基金成立 将为股市新增300亿资金兴业有机增长混合基金23日发布公告称，基金募...,财经
9996,银华基金杨靖聊政策性主题投资机会实录新浪财经讯 银华和谐主题基金拟任基金经理助理杨靖于3月2...,财经
9997,首只基金投资信心指数问世本报讯 (记者吴敏)昨日，嘉实基金宣布推出“嘉实中国基金投资者信心指...,财经
9998,17只阳光私募3月份火速成立证券时报记者 方 丽本报讯 阳光私募产品迎来了发行高潮。WIND...,财经


In [10]:
with open('./data/stop_words_zh.txt', 'r', encoding='utf8') as f:
    stopwords = set(f.read().splitlines())

In [11]:
punctuation = string.punctuation + """！？｡＂＃＄％＆＇（）＊＋，－／：；。＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘'‛“”„‟…‧﹏."""

In [14]:
content = train.content.iloc[0]
content

'马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳，国奥队依然没有摆脱雨水的困扰。7月31日下午6点，国奥队的日常训练再度受到大雨的干扰，无奈之下队员们只慢跑了25分钟就草草收场。31日上午10点，国奥队在奥体中心外场训练的时候，天就是阴沉沉的，气象预报显示当天下午沈阳就有大雨，但幸好队伍上午的训练并没有受到任何干扰。下午6点，当球队抵达训练场时，大雨已经下了几个小时，而且丝毫没有停下来的意思。抱着试一试的态度，球队开始了当天下午的例行训练，25分钟过去了，天气没有任何转好的迹象，为了保护球员们，国奥队决定中止当天的训练，全队立即返回酒店。在雨中训练对足球队来说并不是什么稀罕事，但在奥运会即将开始之前，全队变得“娇贵”了。在沈阳最后一周的训练，国奥队首先要保证现有的球员不再出现意外的伤病情况以免影响正式比赛，因此这一阶段控制训练受伤、控制感冒等疾病的出现被队伍放在了相当重要的位置。而抵达沈阳之后，中后卫冯萧霆就一直没有训练，冯萧霆是7月27日在长春患上了感冒，因此也没有参加29日跟塞尔维亚的热身赛。队伍介绍说，冯萧霆并没有出现发烧症状，但为了安全起见，这两天还是让他静养休息，等感冒彻底好了之后再恢复训练。由于有了冯萧霆这个例子，因此国奥队对雨中训练就显得特别谨慎，主要是担心球员们受凉而引发感冒，造成非战斗减员。而女足队员马晓旭在热身赛中受伤导致无缘奥运的前科，也让在沈阳的国奥队现在格外警惕，“训练中不断嘱咐队员们要注意动作，我们可不能再出这样的事情了。”一位工作人员表示。从长春到沈阳，雨水一路伴随着国奥队，“也邪了，我们走到哪儿雨就下到哪儿，在长春几次训练都被大雨给搅和了，没想到来沈阳又碰到这种事情。”一位国奥球员也对雨水的“青睐”有些不解。'

In [15]:
re.sub(r'[{}]'.format(punctuation), ' ', content)

'马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳 国奥队依然没有摆脱雨水的困扰 7月31日下午6点 国奥队的日常训练再度受到大雨的干扰 无奈之下队员们只慢跑了25分钟就草草收场 31日上午10点 国奥队在奥体中心外场训练的时候 天就是阴沉沉的 气象预报显示当天下午沈阳就有大雨 但幸好队伍上午的训练并没有受到任何干扰 下午6点 当球队抵达训练场时 大雨已经下了几个小时 而且丝毫没有停下来的意思 抱着试一试的态度 球队开始了当天下午的例行训练 25分钟过去了 天气没有任何转好的迹象 为了保护球员们 国奥队决定中止当天的训练 全队立即返回酒店 在雨中训练对足球队来说并不是什么稀罕事 但在奥运会即将开始之前 全队变得 娇贵 了 在沈阳最后一周的训练 国奥队首先要保证现有的球员不再出现意外的伤病情况以免影响正式比赛 因此这一阶段控制训练受伤 控制感冒等疾病的出现被队伍放在了相当重要的位置 而抵达沈阳之后 中后卫冯萧霆就一直没有训练 冯萧霆是7月27日在长春患上了感冒 因此也没有参加29日跟塞尔维亚的热身赛 队伍介绍说 冯萧霆并没有出现发烧症状 但为了安全起见 这两天还是让他静养休息 等感冒彻底好了之后再恢复训练 由于有了冯萧霆这个例子 因此国奥队对雨中训练就显得特别谨慎 主要是担心球员们受凉而引发感冒 造成非战斗减员 而女足队员马晓旭在热身赛中受伤导致无缘奥运的前科 也让在沈阳的国奥队现在格外警惕  训练中不断嘱咐队员们要注意动作 我们可不能再出这样的事情了  一位工作人员表示 从长春到沈阳 雨水一路伴随着国奥队  也邪了 我们走到哪儿雨就下到哪儿 在长春几次训练都被大雨给搅和了 没想到来沈阳又碰到这种事情  一位国奥球员也对雨水的 青睐 有些不解 '

In [16]:
def clean_txt(txt):
    tokens = list(jieba.cut(txt))
    tokens = [token for token in tokens if token.strip() and token not in stopwords]
    return tokens

In [17]:
train['tokens'] = train['content'].apply(clean_txt)
test['tokens'] = test['content'].apply(clean_txt)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\hp\AppData\Local\Temp\jieba.cache
Loading model cost 1.635 seconds.
Prefix dict has been built successfully.


In [18]:
train.to_csv('./cache/train.csv')
test.to_csv('./cache/test.csv')

In [19]:
train

,content,label,tokens
0,马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳，国奥队依然没有...,体育,"[马晓旭, 意外, 受伤, 国奥, 警惕, 无奈, 大雨, 格外, 青睐, 殷家, 军, 记..."
1,商瑞华首战复仇心切 中国玫瑰要用美国方式攻克瑞典多曼来了，瑞典来了，商瑞华首战求3分的信心也...,体育,"[商瑞华, 首战, 复仇, 心切, 中国, 玫瑰, 美国, 方式, 攻克, 瑞典, 多曼来,..."
2,冠军球队迎新欢乐派对 黄旭获大奖张军赢下PK赛新浪体育讯12月27日晚，“冠军高尔夫球队迎新...,体育,"[冠军, 球队, 迎新, 欢乐, 派对, 黄旭获, 大奖, 张军, 赢, 下, PK, 赛,..."
3,辽足签约危机引注册难关 高层威逼利诱合同笑里藏刀新浪体育讯2月24日，辽足爆发了集体拒签风波...,体育,"[辽足, 签约, 危机, 引, 注册, 难关, 高层, 威逼利诱, 合同, 笑里藏刀, 新浪..."
4,揭秘谢亚龙被带走：总局电话骗局 复制南杨轨迹体坛周报特约记者张锐北京报道 谢亚龙已经被公安...,体育,"[揭秘, 谢亚龙, 带走, 总局, 电话, 骗局, 复制, 南杨, 轨迹, 体坛周报, 特约..."
...,...,...,...
49995,打好投资的决胜局□国泰基金 阿邰就长期而言，资产配置占投资成功与否的决定因素高达90%以上。...,财经,"[好, 投资, 决胜局, □, 国泰, 基金, 邰, 长期, 资产, 配置, 占, 投资, ..."
49996,昔明星基金今年首月负收益 QDII成今年胸闷基金□晨报记者 陈重博2009年以来，所有偏股型...,财经,"[昔, 明星, 基金, 今年, 首月, 负, 收益, QDII, 成, 今年, 胸闷, 基金..."
49997,沪基指全周大涨8.25% 创两个月最大周涨幅全景网2月6日讯 受到A股市场牛年高歌猛进影响，...,财经,"[沪, 基指全, 周大涨, 8.25%, 创, 两个, 月, 最大, 周, 涨幅, 全景网,..."
49998,沪基指半日涨2.01% 两市封基近乎全线上扬全景网2月6日讯 沪深基金指数周五早盘大幅收高，...,财经,"[沪, 基指, 半日, 涨, 2.01%, 两市, 封基, 近乎, 全线, 上扬, 全景网,..."


In [20]:
train.head(3)

,content,label,tokens
0,马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳，国奥队依然没有...,体育,"[马晓旭, 意外, 受伤, 国奥, 警惕, 无奈, 大雨, 格外, 青睐, 殷家, 军, 记..."
1,商瑞华首战复仇心切 中国玫瑰要用美国方式攻克瑞典多曼来了，瑞典来了，商瑞华首战求3分的信心也...,体育,"[商瑞华, 首战, 复仇, 心切, 中国, 玫瑰, 美国, 方式, 攻克, 瑞典, 多曼来,..."
2,冠军球队迎新欢乐派对 黄旭获大奖张军赢下PK赛新浪体育讯12月27日晚，“冠军高尔夫球队迎新...,体育,"[冠军, 球队, 迎新, 欢乐, 派对, 黄旭获, 大奖, 张军, 赢, 下, PK, 赛,..."


In [21]:
# 特征数
N = 2048

In [22]:
def identity_tokenizer(text):
    return text

tfidf = TfidfVectorizer(tokenizer=identity_tokenizer, stop_words=None, lowercase=False, max_features=N)    
train_features = tfidf.fit_transform(train.tokens).toarray()
test_features = tfidf.transform(test.tokens).toarray()

In [26]:
from sklearn.linear_model import LogisticRegression

In [24]:
from sklearn import metrics

In [27]:
X_train, y_train = train_features, train.label.values
X_test, y_test = test_features, test.label.values

model = LogisticRegression()
model.fit(X_train, y_train)

f:\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [29]:
y_test_pred = model.predict(X_test)
print(metrics.classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

          体育       1.00      1.00      1.00      1000
          娱乐       0.98      0.98      0.98      1000
          家居       0.96      0.77      0.85      1000
          房产       0.89      0.91      0.90      1000
          教育       0.95      0.91      0.93      1000
          时尚       0.90      0.96      0.93      1000
          时政       0.91      0.96      0.93      1000
          游戏       0.96      0.97      0.97      1000
          科技       0.93      0.97      0.95      1000
          财经       0.95      0.99      0.97      1000

    accuracy                           0.94     10000
   macro avg       0.94      0.94      0.94     10000
weighted avg       0.94      0.94      0.94     10000



In [30]:
train

,content,label,tokens
0,马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳，国奥队依然没有...,体育,"[马晓旭, 意外, 受伤, 国奥, 警惕, 无奈, 大雨, 格外, 青睐, 殷家, 军, 记..."
1,商瑞华首战复仇心切 中国玫瑰要用美国方式攻克瑞典多曼来了，瑞典来了，商瑞华首战求3分的信心也...,体育,"[商瑞华, 首战, 复仇, 心切, 中国, 玫瑰, 美国, 方式, 攻克, 瑞典, 多曼来,..."
2,冠军球队迎新欢乐派对 黄旭获大奖张军赢下PK赛新浪体育讯12月27日晚，“冠军高尔夫球队迎新...,体育,"[冠军, 球队, 迎新, 欢乐, 派对, 黄旭获, 大奖, 张军, 赢, 下, PK, 赛,..."
3,辽足签约危机引注册难关 高层威逼利诱合同笑里藏刀新浪体育讯2月24日，辽足爆发了集体拒签风波...,体育,"[辽足, 签约, 危机, 引, 注册, 难关, 高层, 威逼利诱, 合同, 笑里藏刀, 新浪..."
4,揭秘谢亚龙被带走：总局电话骗局 复制南杨轨迹体坛周报特约记者张锐北京报道 谢亚龙已经被公安...,体育,"[揭秘, 谢亚龙, 带走, 总局, 电话, 骗局, 复制, 南杨, 轨迹, 体坛周报, 特约..."
...,...,...,...
49995,打好投资的决胜局□国泰基金 阿邰就长期而言，资产配置占投资成功与否的决定因素高达90%以上。...,财经,"[好, 投资, 决胜局, □, 国泰, 基金, 邰, 长期, 资产, 配置, 占, 投资, ..."
49996,昔明星基金今年首月负收益 QDII成今年胸闷基金□晨报记者 陈重博2009年以来，所有偏股型...,财经,"[昔, 明星, 基金, 今年, 首月, 负, 收益, QDII, 成, 今年, 胸闷, 基金..."
49997,沪基指全周大涨8.25% 创两个月最大周涨幅全景网2月6日讯 受到A股市场牛年高歌猛进影响，...,财经,"[沪, 基指全, 周大涨, 8.25%, 创, 两个, 月, 最大, 周, 涨幅, 全景网,..."
49998,沪基指半日涨2.01% 两市封基近乎全线上扬全景网2月6日讯 沪深基金指数周五早盘大幅收高，...,财经,"[沪, 基指, 半日, 涨, 2.01%, 两市, 封基, 近乎, 全线, 上扬, 全景网,..."


In [76]:
train = train_bk.copy()

In [77]:
np.random.seed(1234)

In [109]:
dfs = []
n = 2
for label, group in train.groupby('label'):
    group = group.sample(n=n, replace=False)
    dfs.append(group)

In [110]:
train_small = pd.concat(dfs, axis=0)

In [111]:
train_small.head(3)

,content,label,tokens
3922,斯科拉：塔比特有成伟大中锋的基础 我会给他建议新浪体育讯北京时间2月26日消息，火箭球员路易...,体育,"[斯科拉, 塔比, 特有, 成, 伟大, 中锋, 基础, 我会, 建议, 新浪, 体育讯, ..."
2284,三分王对决两双王技高一筹 马修斯36分创生涯新高新浪体育讯北京时间1月8日消息，据美联社报道...,体育,"[三分王, 对决, 两双, 王, 技高一筹, 马修斯, 36, 分创, 生涯, 新高, 新浪..."
6476,甄子丹《十月围城》扮相曝光 三小时完成造型新浪娱乐讯 由陈可辛、黄建新联合监制，陈德森导演的...,娱乐,"[甄子丹, 十月, 围城, 扮相, 曝光, 三, 小时, 完成, 造型, 新浪, 娱乐, 讯..."


In [112]:
tfidf = TfidfVectorizer(tokenizer=identity_tokenizer, stop_words=None, lowercase=False, max_features=N)    
train_features = tfidf.fit_transform(train_small.tokens).toarray()
test_features = tfidf.transform(test.tokens).toarray()

In [113]:
X_train, y_train = train_features, train_small.label.values
X_test, y_test = test_features, test.label.values

model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [114]:
X_train.shape

(20, 2048)

In [115]:
y_test_pred = model.predict(X_test)
print(metrics.classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

          体育       0.58      0.97      0.73      1000
          娱乐       0.67      0.70      0.68      1000
          家居       0.30      0.08      0.12      1000
          房产       0.64      0.62      0.63      1000
          教育       0.60      0.63      0.61      1000
          时尚       0.49      0.80      0.61      1000
          时政       0.80      0.23      0.36      1000
          游戏       0.72      0.76      0.74      1000
          科技       0.04      0.01      0.02      1000
          财经       0.42      0.75      0.54      1000

    accuracy                           0.55     10000
   macro avg       0.53      0.55      0.50     10000
weighted avg       0.53      0.55      0.50     10000



In [117]:
test

,content,label,tokens
0,鲍勃库西奖归谁属？ NCAA最强控卫是坎巴还是弗神新浪体育讯如今，本赛季的NCAA进入到了末...,体育,"[鲍勃, 库西, 奖归, 属, NCAA, 最强, 控卫, 坎巴, 弗神, 新浪, 体育讯,..."
1,麦基砍28+18+5却充满寂寞 纪录之夜他的痛阿联最懂新浪体育讯上天对每个人都是公平的，贾维...,体育,"[麦基, 砍, 28, +, 18, +, 5, 却, 充满, 寂寞, 纪录, 之夜, 痛,..."
2,黄蜂vs湖人首发：科比冲击七连胜 火箭两旧将登场新浪体育讯北京时间3月28日，NBA常规赛洛...,体育,"[黄蜂, vs, 湖人, 首发, 科比, 冲击, 七, 连胜, 火箭, 两旧, 登场, 新浪..."
3,双面谢亚龙作秀终成做作 谁来为低劣行政能力埋单是谁任命了谢亚龙？谁放纵了谢亚龙？谁又该为谢亚...,体育,"[双面, 谢亚龙, 作秀, 终成, 做作, 低劣, 行政, 能力, 埋单, 任命, 谢亚龙,..."
4,兔年首战山西换帅后有虎胆 张学文用乔丹名言励志今晚客场挑战浙江稠州银行队，是山西汾酒男篮的兔...,体育,"[兔年, 首战, 山西, 换帅, 后, 虎胆, 张学文, 乔丹, 名言, 励志, 今晚, 客..."
...,...,...,...
9995,近期18只偏股基金成立 将为股市新增300亿资金兴业有机增长混合基金23日发布公告称，基金募...,财经,"[近期, 18, 只, 偏股, 基金, 成立, 股市, 新增, 300, 亿, 资金, 兴业..."
9996,银华基金杨靖聊政策性主题投资机会实录新浪财经讯 银华和谐主题基金拟任基金经理助理杨靖于3月2...,财经,"[银华, 基金, 杨靖聊, 政策性, 主题, 投资, 机会, 实录, 新浪, 财经, 讯, ..."
9997,首只基金投资信心指数问世本报讯 (记者吴敏)昨日，嘉实基金宣布推出“嘉实中国基金投资者信心指...,财经,"[首, 只, 基金, 投资, 信心, 指数, 问世, 本报讯, (, 记者, 吴敏, ), ..."
9998,17只阳光私募3月份火速成立证券时报记者 方 丽本报讯 阳光私募产品迎来了发行高潮。WIND...,财经,"[17, 只, 阳光, 私募, 3, 月份, 火速, 成立, 证券时报, 记者, 方, 丽,..."


In [118]:
from gensim.models import Word2Vec

f:\python\python37\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [143]:
train_small = train.copy()

In [144]:
# 词向量维度
VEC_SIZE = 128

w2v_model = Word2Vec(
    train_small.content.values, 
    vector_size=VEC_SIZE, 
    min_count=3, 
    seed=1234
)

In [152]:
tfidf = TfidfVectorizer(tokenizer=identity_tokenizer, stop_words=None, lowercase=False, max_features=N) 
tfidf.fit(train_small.tokens)
tfidf_id_to_tokens = {v: k for k, v in tfidf_model.vocabulary_.items()}

f:\python\python37\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  "The parameter 'token_pattern' will not be used"


TfidfVectorizer(lowercase=False, max_features=2048,
                tokenizer=<function identity_tokenizer at 0x000001F30FED5318>)

In [145]:
tokens = train.tokens.values[0]
print(tokens)

['马晓旭', '意外', '受伤', '国奥', '警惕', '无奈', '大雨', '格外', '青睐', '殷家', '军', '记者', '傅亚雨', '沈阳', '报道', '来到', '沈阳', '国奥队', '依然', '没有', '摆脱', '雨水', '困扰', '7', '月', '31', '日', '下午', '6', '点', '国奥队', '日常', '训练', '再度', '受到', '大雨', '干扰', '无奈', '之下', '队员', '只', '慢跑', '25', '分钟', '草草收场', '31', '日', '上午', '10', '点', '国奥队', '奥体中心', '外场', '训练', '天', '阴沉沉', '气象预报', '显示', '当天', '下午', '沈阳', '大雨', '幸好', '队伍', '上午', '训练', '没有', '受到', '干扰', '下午', '6', '点', '球队', '抵达', '训练场', '时', '大雨', '已经', '下', '几个', '小时', '丝毫', '没有', '停下来', '意思', '抱', '试一试', '态度', '球队', '当天', '下午', '例行', '训练', '25', '分钟', '过去', '天气', '没有', '转好', '迹象', '保护', '球员', '国奥队', '决定', '中止', '当天', '训练', '全队', '立即', '返回', '酒店', '雨', '中', '训练', '足球队', '来说', '不是', '稀罕', '事', '奥运会', '即将', '之前', '全队', '变得', '娇贵', '沈阳', '最后', '一周', '训练', '国奥队', '保证', '现有', '球员', '不再', '出现意外', '伤病', '情况', '影响', '正式', '比赛', '这一', '阶段', '控制', '训练', '受伤', '控制', '感冒', '疾病', '出现', '队伍', '放在', '相当', '重要', '位置', '抵达', '沈阳', '之后', '中', '后卫', '冯萧霆', '一直', '没有', '训练', '冯萧霆', '7', '月', '

In [187]:
def build_feature(tokens):
    tfidf_feature = tfidf_model.transform([tokens])
    tfidf_feature = dict(tfidf_feature.todok().items())
    tfidf_feature = {k[1]: v for k, v in tfidf_feature.items()}
    
    feature = np.zeros(VEC_SIZE)
    d = 0
    # 对于每一个词
    for token_id, tfidf_weight in tfidf_feature.items():
        # 该词必须要有词向量，如果没有就跳过该词
        token = tfidf_id_to_tokens[token_id]
        if token not in w2v_model.wv:
            continue
        vector = w2v_model.wv[token]
        feature += tfidf_weight * vector
        d += tfidf_weight
    if d:
        feature /= d
    return feature

In [146]:
def to_feature(tokens):
    feature = np.zeros(VEC_SIZE)
    n = 0
    for token in tokens:
        if token in w2v_model.wv:
            feature += w2v_model.wv[token]
            n += 1
    if n:
        feature /= n
    return feature

In [195]:
to_feature = build_feature

In [196]:
train_small['feature'] = train_small.tokens.apply(to_feature)
X_train = np.vstack(train_small.feature.values)

In [197]:
test['feature'] = test.tokens.apply(to_feature)
X_test = np.vstack(test.feature.values)

In [205]:
y_train = train_small.label.values
y_test = test.label.values

In [208]:
indices = np.random.choice(len(y_train), 100, replace=False)

In [212]:
indices = defaultdict(list)
for index, label in enumerate(y_train):
    indices[label].append(index)

In [221]:
n = 100
selected_indices = []
for label, label_indices in indices.items():
    selected_indices.extend(np.random.choice(label_indices, n, replace=False))

In [222]:
model = LogisticRegression()
model.fit(X_train[selected_indices], y_train[selected_indices])

f:\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [223]:
y_test_pred = model.predict(X_test)
print(metrics.classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

          体育       0.78      0.74      0.76      1000
          娱乐       0.71      0.64      0.67      1000
          家居       0.43      0.08      0.13      1000
          房产       0.42      0.48      0.45      1000
          教育       0.40      0.50      0.44      1000
          时尚       0.73      0.71      0.72      1000
          时政       0.54      0.56      0.55      1000
          游戏       0.45      0.54      0.49      1000
          科技       0.61      0.71      0.65      1000
          财经       0.44      0.53      0.48      1000

    accuracy                           0.55     10000
   macro avg       0.55      0.55      0.53     10000
weighted avg       0.55      0.55      0.53     10000



In [151]:
y_test_pred = model.predict(X_test)
print(metrics.classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

          体育       0.82      0.85      0.84      1000
          娱乐       0.77      0.79      0.78      1000
          家居       0.56      0.16      0.24      1000
          房产       0.53      0.60      0.56      1000
          教育       0.50      0.56      0.53      1000
          时尚       0.77      0.85      0.80      1000
          时政       0.64      0.67      0.65      1000
          游戏       0.58      0.60      0.59      1000
          科技       0.71      0.75      0.73      1000
          财经       0.62      0.73      0.67      1000

    accuracy                           0.66     10000
   macro avg       0.65      0.66      0.64     10000
weighted avg       0.65      0.66      0.64     10000

